In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'pandas'

In [21]:
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(6)
tf.random.set_seed(6)

In [12]:
train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [13]:
X_train = train.copy()
y_train = X_train.pop('label')

Scale the data

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

One hot encode target

In [16]:
y_train = pd.get_dummies(y_train)   

Model

In [28]:
keras.__version__

NameError: name 'keras' is not defined

In [2]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Resizing, RandomZoom, RandomRotation
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'keras'

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=6)

2021-12-08 12:52:07.197878: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [25]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

In [26]:
X_train = np.array(X_train).reshape((31500, 28, 28, 1))
X_val = np.array(X_val).reshape((10500, 28, 28, 1))
X_test = np.array(X_test).reshape((28000, 28, 28, 1))

In [41]:
lr_plat = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.3, min_lr=0.00001)

In [42]:
inp = Input(shape=(28, 28, 1))      

# Data augmentation
resized = Resizing(38,38)(inp)
zoom = RandomZoom(0.07)(resized)   
rot = RandomRotation(0.07)(zoom)

conv = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(rot)                             
pool = MaxPooling2D()(conv)          
norm = BatchNormalization()(pool)

conv_2 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(norm)
pool_2 = MaxPooling2D()(conv_2)
norm_2 = BatchNormalization()(pool_2)

conv_3 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(norm_2)        
pool_3 = MaxPooling2D()(conv_3)
norm_3 = BatchNormalization()(pool_3)

conv_4 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(norm_3)        
pool_4 = MaxPooling2D()(conv_4)
norm_4 = BatchNormalization()(pool_4)

conv_5 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(norm_4)        
pool_5 = MaxPooling2D()(conv_5)
norm_5 = BatchNormalization()(pool_5)

hidden = Dense(256, activation='relu')(norm_5)
flatten = Flatten()(hidden)
out = Dense(10, activation='softmax')(flatten)

model = Model(inputs=inp, outputs=out)

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=40, callbacks=[early_stop, lr_plat], batch_size=64)                                                         

Epoch 1/40
493/493 [==============================] - 56s 112ms/step - loss: 0.2259 - accuracy: 0.9329 - val_loss: 0.3258 - val_accuracy: 0.8806
Epoch 2/40
493/493 [==============================] - 58s 117ms/step - loss: 0.0797 - accuracy: 0.9755 - val_loss: 0.0623 - val_accuracy: 0.9813
Epoch 3/40
493/493 [==============================] - 58s 117ms/step - loss: 0.0562 - accuracy: 0.9816 - val_loss: 0.0495 - val_accuracy: 0.9852
Epoch 4/40
493/493 [==============================] - 58s 117ms/step - loss: 0.0491 - accuracy: 0.9848 - val_loss: 0.0504 - val_accuracy: 0.9848
Epoch 5/40
493/493 [==============================] - 57s 116ms/step - loss: 0.0442 - accuracy: 0.9862 - val_loss: 0.0518 - val_accuracy: 0.9853
Epoch 6/40
493/493 [==============================] - 58s 117ms/step - loss: 0.0376 - accuracy: 0.9884 - val_loss: 0.0491 - val_accuracy: 0.9850

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 7/40
493/493 [=============================

In [44]:
pred = model.predict(X_test)
pred_classes = pred.argmax(axis=-1)
sub['Label'] = pred_classes              